In [1]:
# Импорт Библиотек
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from lxml import html

### Вспомогательные функции

In [2]:
# Удаление лишних символов и замена запятой на точку
def wrong_string_to_float(new_string):
    try:
        new_string = new_string.replace(u'\x0a', u'')
        new_string = new_string.replace(u'\x0d', u'')
        new_string = new_string.replace('руб', u'')
        new_string = re.sub(r"\s+", "", new_string, flags=re.UNICODE)
        new_string = new_string.replace(',', '.')
        price = float(new_string)
    except:
        price = None
    return price

In [3]:
def save_parsed_data_to_file(list_parsed_url_page_catalog, list_goods):
    
    # Запись списка спарсеных страниц каталога
    df_parsed_url = pd.DataFrame(list_parsed_url_page_catalog)
    df_parsed_url.columns = ['parsed_url']
    df_parsed_url.to_csv('./parsed_url.csv', index=False)
                  
    # Запись списка спарсеных товаров
    columns = ['name', 'code_number', 'article_number', 'manufacturer', 'description', 'price']
    df_parsed_goods = pd.DataFrame(list_goods)
    df_parsed_goods.columns = columns
    df_parsed_goods.to_csv('./parsed_goods.csv', header = columns, index=False)
    print('Сохранено успешно')
                  

In [4]:
# Печать списка товаров на странице
def print_goods_on_page(soup):
    page = soup.findAll('div', class_='center_block')
    num = 0
    for i in page:
        #print(i)
        name = i.findAll('p', class_='product_desc')
        for cur in name:
            print (num, cur.get_text())
            num += 1
            print('='*70)

### Функция для получения списка страниц каталога товаров для парсинга

In [5]:
# Парсинг ссылок на страницы каталога товаров на сайте
def get_list_url_page_catalog(soup):
    list_url_page = []
    page = soup.findAll('div', class_='block_content')
    for cur_page in page:
        name = cur_page.findAll('li')
        for cur in name:
            urlu = cur.findAll('a')
            for ppp in urlu:
                page_url = ppp.get('href')
                #print(page_url)
                list_url_page.append(page_url)
    #print(list_url_page)
    print('Длинна Списка страниц для парсинга = ', len(list_url_page))    
    return list_url_page

### Функции для получения списка товаров на странице, их цен и характеристик

In [6]:
# Получение обьекта soup страницe каталога которую будем парсить по ссылке url
def get_soup_from_url(url):
    page = requests.get(url)
    #print(page.status_code)
    soup = BeautifulSoup(page.text, "html.parser")
    return soup

In [7]:
# Получение ссылок на товары со страницы
def get_list_goods_url_from_page(soup):
    list_goods_url = []
    page = soup.findAll('div', class_='center_block')
    for i in page:
        name = i.findAll('a', class_='product_img_link')
        for cur in name:
            list_goods_url.append(cur.get('href'))
    print('Количество товаров на странице для парсинга = ',len(list_goods_url))
    return list_goods_url

In [8]:
# Получение атрибутов товара по ссылке
def gef_get_goods_details(goods_url):
    product         = []
    name            = None
    code_number     = None
    article_number  = None
    manufacturer    = None
    description     = None
    price           = None
    
    try:
        local_page = requests.get(goods_url)
        descr_page = BeautifulSoup(local_page.text, "html.parser")

        #Получение наименования товара по ссылке страницы
        code_contents = descr_page.findAll('div', class_="rte align_justify")
        for i in code_contents:
            name = i.get_text()
        #    print('Наименование  - ', name)

        #Получение кода товара по ссылке страницы
        code_contents = descr_page.findAll('div', id="pb-left-column")
        for i in code_contents:
            code = descr_page.findAll('h1')
        for i in code:
            code_number = i.get_text()
        #print('Код Товара      - ', code_number)

        #Получение артикула и производителя товара по ссылке страницы
        articul = descr_page.findAll('span', class_='editable')
        list_info = []
        for i in articul:
            list_info.append(i.get_text()) 
        article_number = list_info[0]
        manufacturer   = list_info[1].replace(u'\x0a', u'')
        #print('Артикул       - ', article_number)
        #print('Производитель - ', manufacturer)

        #Получение описания товара по ссылке страницы
        descr = descr_page.findAll('div', class_='rte', id="idTab1")
        for i in descr:
            description = i.get_text()
        #    print('Описание      - ', description)

        #Получение цены товара по ссылке страницы
        price_contents = descr_page.findAll('span', id="our_price_display")
        for i in price_contents:
            price = wrong_string_to_float(i.get_text())
        #    print('Цена          -', price)
        #print('='*70)
    except:
        pass
    product.append(name)
    product.append(code_number)
    product.append(article_number)
    product.append(manufacturer)
    product.append(description)
    product.append(price)
    
    return product
        
    

### Итоговая сборка парсера

In [9]:
# Ссылка на сайт который парсим
url = 'http://service.ru/'
# Получаем HTML страницу по ссылке
page = requests.get(url)
# Получение из HTML страницы обьекта типа soup для разбора по HTML тегам
soup = BeautifulSoup(page.text, "html.parser")
# Код 200 означает, что сайт откликнулся и выдал нам страницу
print(page.status_code) 

200


In [10]:
list_parsed_url_page_catalog =[] # Список спарсенных страниц каталога
list_goods = []                  # Общий список спрасеных товаров

# Парсинг ссылок на страницы каталога товаров на сайте
list_url_page_catalog  =  get_list_url_page_catalog(soup)

# Отбрасываем последние 4 сыылки как ссылки не на страницы катлога
list_url_page_catalog  = list_url_page_catalog[:2]   # Парсим первые две страницы каталога
#list_url_page_catalog  = list_url_page_catalog[:-4] # Парсим весь каталог товаров на сайте

# Перебор разделов каталога
num_page = 1
for cur_page_catalog in list_url_page_catalog:
    print('Обрабатываю страницу ', num_page, ' из ', len(list_url_page_catalog))
    print('Всего спарсено товаров - ', len(list_goods))
    # Получение товаров из раздела каталога каталога
    soup_page_catalog = get_soup_from_url(cur_page_catalog)
    list_goods_url = get_list_goods_url_from_page(soup_page_catalog)
    num_goods = 1
    # Перебор товаров в разделе каталога
    for cur_goods_url in list_goods_url:
        print('     Обрабатываю товар ', num_goods, ' из ', len(list_goods_url))
        goods_details = gef_get_goods_details(cur_goods_url)
        # добавляем полученный товар в список
        list_goods.append(goods_details)
        num_goods += 1    
    num_page += 1
    list_parsed_url_page_catalog.append(cur_page_catalog)
    save_parsed_data_to_file(list_parsed_url_page_catalog, list_goods)
print('Спарсено товаров', len(list_goods))

Длинна Списка страниц для парсинга =  471
Обрабатываю страницу  1  из  2
Всего спарсено товаров -  0
Количество товаров на странице для парсинга =  150
     Обрабатываю товар  1  из  150
     Обрабатываю товар  2  из  150
     Обрабатываю товар  3  из  150
     Обрабатываю товар  4  из  150
     Обрабатываю товар  5  из  150
     Обрабатываю товар  6  из  150
     Обрабатываю товар  7  из  150
     Обрабатываю товар  8  из  150
     Обрабатываю товар  9  из  150
     Обрабатываю товар  10  из  150
     Обрабатываю товар  11  из  150
     Обрабатываю товар  12  из  150
     Обрабатываю товар  13  из  150
     Обрабатываю товар  14  из  150
     Обрабатываю товар  15  из  150
     Обрабатываю товар  16  из  150
     Обрабатываю товар  17  из  150
     Обрабатываю товар  18  из  150
     Обрабатываю товар  19  из  150
     Обрабатываю товар  20  из  150
     Обрабатываю товар  21  из  150
     Обрабатываю товар  22  из  150
     Обрабатываю товар  23  из  150
     Обрабатываю товар  24  и

In [13]:
# Чтение списка спарсеных разделов каталога и спарсеных товаров
parsed_url_page_catalog = pd.read_csv('./parsed_url.csv')
goods = pd.read_csv('./parsed_goods.csv')
print(parsed_url_page_catalog)
print()
print(goods.head(5))

                                          parsed_url
0  http://service.ru/3-bumaga_i_bumazhnaja_produk...
1               http://service.ru/17-bumaga_ofisnaja

                                                name  code_number  \
0  Конверт С65 (114х229),PostSec, белый, банковск...       A29941   
1         Карточка учёта материалов А5, хром., 97 г.       A07716   
2  Карточка складского учёта материалов А5, хром....       A00973   
3          Карточка учёта материалов А4, хром., 97 г       A03297   
4           Тетрадь школьная, косая лин., офсет,12л.  00-00053528   

  article_number     manufacturer  \
0       121181/П           Постак   
1        М-17/A5           ФЕНИКС   
2           М-17           ФЕНИКС   
3        М-17/А4           ФЕНИКС   
4        С 274/4  ГОЗНАК БЕЛАРУСИ   

                                         description  price  
0  Изготовлены из офсетной бумаги плотностью 80 г...   2.34  
1                     Плотность - 97 г/м2. Формат А5   3.50  
2               

In [14]:
# Таблица спарсенных товаров
goods.head(5)

,name,code_number,article_number,manufacturer,description,price
0,"Конверт С65 (114х229),PostSec, белый, банковск...",A29941,121181/П,Постак,Изготовлены из офсетной бумаги плотностью 80 г...,2.34
1,"Карточка учёта материалов А5, хром., 97 г.",A07716,М-17/A5,ФЕНИКС,Плотность - 97 г/м2. Формат А5,3.50
2,"Карточка складского учёта материалов А5, хром....",A00973,М-17,ФЕНИКС,"Формат А5, плотность - 72 г/м2",3.88
3,"Карточка учёта материалов А4, хром., 97 г",A03297,М-17/А4,ФЕНИКС,Плотность - 97 г/м2. Формат А4,4.18
4,"Тетрадь школьная, косая лин., офсет,12л.",00-00053528,С 274/4,ГОЗНАК БЕЛАРУСИ,"Тетрадь школьная, косая лин., офсет,12л.",4.84


In [ ]:
# Посторочечный вывод спарсенных товаров
for i in list_goods:
    print(i)
    print()